In [3]:
import xarray as xr 
import numpy as np
import skimage
import matplotlib as mpl
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf
#import py_eddy_tracker.data 



In [4]:
ds_eddys = xr.open_dataset(r'E:\Proyecto de Grado\Data\1999_año_niña.nc')
ds_eddys

<xarray.Dataset>
Dimensions:    (depth: 5, latitude: 200, time: 365, longitude: 370)
Coordinates:
  * depth      (depth) float32 0.494 1.541 2.646 3.819 5.078
  * latitude   (latitude) float32 7.0 7.083 7.167 7.25 ... 23.42 23.5 23.58
  * time       (time) datetime64[ns] 1999-01-01T12:00:00 ... 1999-12-31T12:00:00
  * longitude  (longitude) float32 -89.83 -89.75 -89.67 ... -59.25 -59.17 -59.08
Data variables:
    vo         (time, depth, latitude, longitude) float32 ...
    uo         (time, depth, latitude, longitude) float32 ...
Attributes: (12/25)
    title:                              daily mean fields from Global Ocean P...
    easting:                            longitude
    northing:                           latitude
    history:                            2022/05/25 21:54:07 MERCATOR OCEAN Ne...
    source:                             MERCATOR GLORYS12V1
    institution:                        MERCATOR OCEAN
    ...                                 ...
    FROM_ORIGINAL_FILE__longitude_max:  179.91667
    FROM_ORIGINAL_FILE__latitude_min:   -80.0
    FROM_ORIGINAL_FILE__latitude_max:   90.0
    z_min:                              0.494025
    z_max:                              5727.917
    _CoordSysBuilder:                   ucar.nc2.dataset.conv.CF1Convention

In [15]:
i=1

In [19]:
for i in range (32):
    lat = ds_eddys['latitude'].to_numpy()
    lon = ds_eddys['longitude'].to_numpy()
    #Para el cálculo de la aceleración
    V = ds_eddys["vo"].mean(dim=["depth"]).to_numpy()
    U = ds_eddys["uo"].mean(dim=["depth"]).to_numpy()
    v = V["vo", {"time" : i}]
    u = U["uo", {"time" : i}]
    vo = V["vo", {"time" : i-1}]
    uo = U["uo", {"time" : i-1}]

    X, Y = np.meshgrid(lon, lat)

    #x = skimage.measure.block_reduce(X, (10, 10), np.nanmean)
    #y = skimage.measure.block_reduce(Y, (10, 10), np.nanmean)
    #u = skimage.measure.block_reduce(U, (10, 10), np.nanmean)
    #v = skimage.measure.block_reduce(V, (10, 10), np.nanmean)

    # Crea una figura y ejes con la proyección de latitud y longitud
    fig, ax = plt.subplots(1, figsize=(8, 6), subplot_kw={'projection': ccrs.PlateCarree()})
    loc = ticker.MultipleLocator(4)

    # Agrega el mapa base (costas, límites de países, etc.)
    ax.add_feature(cf.COASTLINE)
    ax.add_feature(cf.BORDERS)

    #genera una capa raster
    # Usa origin='lower' para mostrar los datos correctamente
    # Usa vmin=0 y vmax=1.5 para establecer la escala de la barra de colores
    R = ax.imshow((((u["uo"]-uo["uo"])/ds_eddys[{"time" : i}]-ds_eddys[{"time" : i-1}])**2 
                + (((u["vo"]-vo["vo"])/ds_eddys[{"time" : i}]-ds_eddys[{"time" : i-1}])**2)),
                extent=[lon.min(), lon.max(), lat.min(), lat.max()],
                cmap='gist_ncar', origin='lower', vmin=0, vmax=1)

    # Crea el gráfico de vectores
    #Q = ax.quiver(x, y, u, v, (u**2 + v**2), scale=10, color= 'w')

    # Agrega una barra de color asociada a la magnitud de los vectores
    plt.colorbar(R, label=r"Velocity [$\frac{m}{s}$]", shrink = 0.5)

    # Establece los límites del mapa y etiquetas de los ejes
    ax.set(ylim=(lat.min(), lat.max()), xlim=(lon.min(), lon.max()))                                                                # Agrega un marco naval con las coordenadas en rojo y con líneas discontinuas
    ax.gridlines(draw_labels={"bottom": "x", "left": "y"}, xlocs=loc, ylocs=loc)

    # Muestra el mapa
    plt.title(f"Velocidades de Corrientes superficiales de {ds_eddys[{'time' : i}]}")
    plt.show()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices